Name:

NetID:

### Introduction

In this lab, we will investigate two highly useful features for image analysis: color histogram and HOG. Both of these features are widely used in computer vision and image analysis problems of all kinds, such as face recognition, similar image search, image classification, and object detection.

If you are using SageMathCloud, opencv and tensorflow are already installed, you can skip this note.

## Notes on OpenCV

Depending on the OS you are using there might be different instructions on how to build and install OpenCV with Python. We will be using OpenCV 3.1.0 for this lab. Older versions should work as well, but you might have to change some lines of code. Depending on your comfort level, you may choose either pathway.

Make sure to at least get OpenCV working before you leave the lab today. Search online for how to install OpenCV on your OS. Installing from prebuilt binaries are recommended. If you installed successfully, you can run 

\>>> import cv2

\>>> print cv2.\__version\__

**DO NOT SUBMIT ANY OpenCV files.**

# Part 1. Color Histogram

For image analysis, much of the information of an image can be conveyed merely by its histogram, although other, more sofisticated features, such as SIFT, are often used and needed in big data problems. Here, we'll investigate how useful the histogram can be. For this section, you will need to download the Caltech-101 dataset (http://www.vision.caltech.edu/Image_Datasets/Caltech101/). This dataset contains 101 different object categories, each with between 40 to 800 images of size 300 x 200, roughly. (This dataset pales in comparison to other modern, "big data" image datasets, but it's the best that your computer can likely handle easily.)

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
from scipy import misc
import numpy as np

First, we'll look at three test-case images.

In [ ]:
beach = misc.imread('beach.png')
desert1 = misc.imread('desert1.jpg')
desert2 = misc.imread('desert2.jpg')

plt.figure()
plt.subplot(1,3,1)
plt.imshow(beach)
plt.subplot(1,3,2)
plt.imshow(desert1)
plt.subplot(1,3,3)
plt.imshow(desert2)

We provide the following two functions to generate and compare histograms. They are built upon OpenCV's implementation [compareHist](http://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html). We use the CV_COMP_INTERSECT method. 
The image has three channels, the comparison results are stored in x1, x2 and x3. And Euclidean distance is used to give a single output number **comp**. 

##### This function is used to compare two histograms using the histogram intersection

In [ ]:
def compHist(h1, h2):
    # h1, h2 - input histograms to compare, which should be of dimension (3,N) for color images and (1,N) for
    #              grayscale images. If the two histograms are not the same dimensions, the output is a large
    #              value to enforce their disimilarity
    if h1.shape[0] != h2.shape[0]:
        return 100
    else:
        x1 = cv2.compareHist(h1[0,:].astype('float32'), h2[0,:].astype('float32'), 3)
        x2 = cv2.compareHist(h1[1,:].astype('float32'), h2[1,:].astype('float32'), 3)
        x3 = cv2.compareHist(h1[2,:].astype('float32'), h2[2,:].astype('float32'), 3)
        comp = np.sqrt(x1**2 + x2**2 + x3**2)
        return comp

##### This function is used to generates a histogram from an input image.  Notice how we normalize the histogram by dividing by the total pixels in the image. 

In [ ]:


def generateHist(img, n_bins):
    # img - the input image
    # n_bins - the number of bins of the output histogram
    # returns: h - the histogram
    k = len(img.shape)
    if k == 2:
        h = np.zeros((1,n_bins),dtype='float32')
        M,N = img.shape
        h[0,:][:, np.newaxis] = cv2.calcHist([img], [0], None, [n_bins], [0,256])
        h = h/(M*N)
    else:
        h = np.zeros((3,n_bins),dtype='float32')
        M,N,D = img.shape
        h[0,:][:, np.newaxis] = cv2.calcHist([img], [0], None, [n_bins], [0,256])
        h[1,:][:, np.newaxis] = cv2.calcHist([img], [1], None, [n_bins], [0,256])
        h[2,:][:, np.newaxis] = cv2.calcHist([img], [2], None, [n_bins], [0,256])
        h = h/(M*N)
    return h

Now, let's use the above functions to generate histograms for the three test-cases.

In [ ]:
n_bins = 100

hist_desert1 = generateHist(desert1, n_bins)
hist_desert2 = generateHist(desert2, n_bins)
hist_beach = generateHist(beach, n_bins)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=True)
f.set_size_inches((8,8))
ax1.plot(hist_beach[0,:])
ax1.plot(hist_beach[1,:])
ax1.plot(hist_beach[2,:])
ax1.set_title('Histograms of Beach and Desert Scenes')

ax2.plot(hist_desert1[0,:])
ax2.plot(hist_desert1[1,:])
ax2.plot(hist_desert1[2,:])

ax3.plot(hist_desert2[0,:])
ax3.plot(hist_desert2[1,:])
ax3.plot(hist_desert2[2,:])
ax3.set_xlabel('Color Intensity Bins')

Using the method=CV_COMP_INTERSECT interesction as a comparison metric, we compare how "similar" are the histograms. Which two images are the most similar? (Remember that this metric is a distance, so smaller values mean more "similar".)

In [ ]:
print 'Beach vs. Desert1:\t' + str(compHist(hist_beach, hist_desert1)) # beach vs. desert1
print 'Desert2 vs. Desert1:\t' + str(compHist(hist_desert2, hist_desert1)) # desert2 vs. desert1
print 'Beach vs. Desert2:\t' + str(compHist(hist_beach, hist_desert2)) # beach vs. desert2

### Exercise 1:

a. Write your own implementation of the histogram. It should take in an image and a parameter called "n_bins," which is the number of bins for the histogram. (In an image, pixel values range from 0 - 255, but the histogram could have fewer than 255 bins. If, for instance, n_bins = 100, then each bin of the histogram would store between 2-3 different pixel intensity values.) 

For grayscale images, the output will be a 1D vector of dimension (1, n_bins). For color images, compute a 1D histogram of each channel with n_bins and form the three histograms into a 2D matrix of dimension (3, n_bins). The histogram should also be normalized by the total number of pixels, so that along each row it sums to 1. 

**You are NOT allowed to use any OpenCV or third party libraries.** 

In [ ]:
def MygenerateHist(image, n_bins):
    #Your code here
    return hist

b. Write your own implementation of the histogram comparing. See the formula in the Intersection (method=CV_COMP_INTERSECT) OpenCV for how to implement the histogram intersection. 

**You are NOT allowed to use any OpenCV or third party libraries.** 

In [ ]:
def MycompHist(h1, h2):
        #Your code here
        return comp

c. Testing your functions using the following code. 

In [ ]:
hist_desert1 = MygenerateHist(desert1, nbins)
hist_desert2 = MygenerateHist(desert2, nbins)
hist_beach = MygenerateHist(beach, nbins)

print 'Beach vs. Desert1:\t' + str(MycompHist(hist_beach, hist_desert1)) # beach vs. desert1
print 'Desert2 vs. Desert1:\t' + str(MycompHist(hist_desert2, hist_desert1)) # desert2 vs. desert1
print 'Beach vs. Desert2:\t' + str(MycompHist(hist_beach, hist_desert2)) # beach vs. desert2

# Part 2: Similar Image Search by Histogram Matching

**In the news**: Flickr and Pinterest rolled out update last week (March 7th, 2017) to allow users to search for images based on visual similarity. Flickr in their blog post says it uses deep neural networks to analyze the photos as they are uploaded to the server. 

In this lab, using the histogram as a feature for our image, let's see how well it works when searching for similar images in a large dataset. You will need to import the lab5.py file, as done below, to use a function to load the Caltech-101 dataset. You can download from [here](http://www.vision.caltech.edu/Image_Datasets/Caltech101/) and unzipped this dataset to the same directory as your notebook. It should be in a folder called '101_ObjectCategories.'

In [ ]:
import lab5

dataset_dir = '101_ObjectCategories/'

n_images, classes, image_names = lab5.load_image_dataset(dataset_dir)

The following code generates histograms for all of the images in the dataset.

In [ ]:
hists = []

n_bins = 100

for name in image_names:
    temp_image = misc.imread(name)
    temp_hist = generateHist(temp_image, n_bins)
    hists.append(temp_hist)

Here are the four test query images that we will use to search for similar images. Notice the diversity in their histograms.

In [ ]:
# Compare a test histogram to the database
test_ind = [450, 2423, 5211, 8134]
n_bins = 100

for i in test_ind:
    test = misc.imread(image_names[i])
    plt.figure()
    plt.imshow(test)

    test_hist = generateHist(test, n_bins)

    plt.figure()
    plt.plot(test_hist[0,:], color='red')
    plt.plot(test_hist[1,:], color='green')
    plt.plot(test_hist[2,:], color='blue')
    plt.title('Histogram for Image ' + str(i))

### Exercise 2 Finding similar images

The code below iterates through the above test query images and searches for their 10 most similar images (K is the number of similar images in the code). Fill in the missing elements to generate the histograms and compare them using your own histogram implementation from Exercise 1. The number of bins for the histogram should be n_bins = 100.

The images will be stored in a directory called "closest" in the same location as your notebook. They will have suffixes denoting which of the four images they match and their order in terms of similarity. (Note that the first image should be the query image itself, since it is still in the dataset and is more similar to itself than any other image). Submit this folder with your notebook for the lab.

Do the results look reasonable? Explain why or why not.

##### Documentation on [argsort](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html)

In [ ]:
import os

closest_dir = 'closest/'
if not os.path.exists(closest_dir):
    os.makedirs(closest_dir)

n_bins = 100
K = 10
    
for ind in test_ind:
    print 'Finding closest ' + str(K) + ' images to ' + image_names[ind]
    test = misc.imread(image_names[ind])
    
    test_hist = [] # edit this line to make things right

    D = np.zeros(n_images)
    for i in range(n_images):
        D[i] =  0 # edit this line to make things right 
    idx = np.argsort(D)
    
    closest = []
    for i in range(K):
        closest.append(image_names[idx[i]])
        img = misc.imread(closest[i])
        misc.imsave(closest_dir + 'closest_' + str(ind) + '_' + str(i) + '.jpg', img)
        print '\tSaving closest image: ' + image_names[idx[i]]
    

Your answer (within 2 lines): 

# Part 3 Histogram of Oriented Gradients

We will learn the details of the Histogram of Oriented Gradients (HOG) feature descriptor. You are strongly encouraged to read this blog first: http://www.learnopencv.com/histogram-of-oriented-gradients/. It contains the details of the implementations. First, let's see the HOG feature by OpenCV. 

In [ ]:
import cv2

hog = cv2.HOGDescriptor()
im = cv2.imread('Duncan.jpg')

h = hog.compute(im)  # HoG feature by OpenCV
plt.plot(h)

im = np.fliplr(im.reshape(-1,3)).reshape(im.shape)  #convert BGR to RGB

Now we are going to calculate the HOG feature of the above image patch by ourself. You are allowed to use OpenCV functions but are not allowed to call HOG descriptor directly. 

**Step 1**: Calculate the Gradient Images. After we obtained the gradients the RGB channels, we just pick the channel with the strongest gradient magnitude. After this step, you should get the magnitude and direction of the gradient for each pixel, both of which are a 128\*64 matrix. 

In [ ]:
im = np.float32(im)/255.0
gx = cv2.Sobel(im, cv2.CV_32F, 1, 0, ksize=1)
gy = cv2.Sobel(im, cv2.CV_32F, 0, 1, ksize=1)

#plot the X and Y gradients.
plt.figure()
plt.subplot(1,3,1)
plt.imshow(im)
plt.subplot(1,3,2)
plt.imshow(np.abs(gx))
plt.subplot(1,3,3)
plt.imshow(np.abs(gy))

# Python Calculate gradient magnitude and direction ( in degrees ) 
mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)

### Exercise 3:

Pick the channel with the strongest gradient magnitude for each pixel. Convert the degree from [0,360] to [0,180] by ignoring the sign of the gradient direction.

In [ ]:
#your code here. 


#plot your result
plt.figure()
plt.subplot(1,2,1)
plt.imshow(mag, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(angle, cmap='gray')

### Exercise 4:

**Step 2**: Calculate Histogram of Gradients in 8×8 cells.

In this step, the image is divided into 8×8 cells and a histogram of gradients is calculated for each 8×8 cells. We create 9 bins, i.e. [0,20,40,60,80,100,120,140,160]. You will need to add the magnitude to the corresponding bin based on the direction. You should use interpolation. For example, 30 degree should be divided into half in 20 and half in 40. After this step, we should get 8\*16 cells, and each cell has a histogram of gradients with 9 bins.   

In [ ]:
cell_size = 8
nbins = 9

nH= im.shape[0]/cell_size 
nW = im.shape[1]/cell_size

raw_hog = np.zeros((nH,nW,nbins),dtype=np.float32)   # define the histogram of gradients 

# Your code here




# plot this cell for the TA to check correctness.         
plt.plot(raw_hog[5,5,:])

### Exercise 5:

**Step 3**: Block Normalization

In this step, we will need to normlize the HOG features in every 16\*16 blocks, which is every 4 cells. In each block there will be a 36\*1 feature vector, which is normalized by its L2 norm. The window is then moved by 1 cell. So finally, we will have the 7\*15 blocks with normalized features. We reshape them into one vector.  

In [ ]:
hog = np.zeros((nH-1,nW-1,nbins*4),dtype=np.float32)

# Your code here





        
hog = np.reshape(hog,[-1])


We compare your implementation and the OpenCV implementation. They may not be the same. Think about why.

In [ ]:
plt.figure()
plt.plot(hog)
plt.figure()
plt.plot(h)


### Exercise 6:  

Extra credit (up to 30 points). Implement a pedestain detector based on your HOG features. You can download the INRIA Person Dataset for training and testing http://pascal.inrialpes.fr/data/human/. You can train a SVM to classify the neg and pos examples based on their HOG features. Print your test accuracy. And plot 3 images of true detect, 3 images of false detect, and 3 images of miss detect. You can use the OpenCV SVM package directly. 